In [1]:
import itertools
import pandas as pd

# Read Test Data
PR2  : (mixed, base, noBase)  
C : (mixed, base, noBase)

In [2]:
# PR2_TYPE = ["mixed", "base", "noBase"]
# C_TYPE = ["mixed", "base", "noBase"]

PR2_TYPE = ["mixed", "base", "noBase"]
C_TYPE = ["mixed", "base", "noBase"]

test_lst = [i for i in itertools.product(PR2_TYPE, C_TYPE)]
test_data_lst = []
test_data_name_lst = []
for i in test_lst:
    _test_data_filename = f'df_{i[0]}_{i[1]}.csv'
    _df = pd.read_csv(f'./gen_data/{_test_data_filename}')
    test_data_lst.append(_df)
    test_data_name_lst.append(_test_data_filename)
    print(f'Read {_test_data_filename}, Shape : {_df.shape}')
print(f'Test Length : {len(test_data_name_lst)}\n{test_data_name_lst}')

Read df_mixed_mixed.csv, Shape : (72, 5)
Read df_mixed_base.csv, Shape : (24, 5)
Read df_mixed_noBase.csv, Shape : (48, 5)
Read df_base_mixed.csv, Shape : (24, 5)
Read df_base_base.csv, Shape : (8, 5)
Read df_base_noBase.csv, Shape : (16, 5)
Read df_noBase_mixed.csv, Shape : (48, 5)
Read df_noBase_base.csv, Shape : (16, 5)
Read df_noBase_noBase.csv, Shape : (32, 5)
Test Length : 9
['df_mixed_mixed.csv', 'df_mixed_base.csv', 'df_mixed_noBase.csv', 'df_base_mixed.csv', 'df_base_base.csv', 'df_base_noBase.csv', 'df_noBase_mixed.csv', 'df_noBase_base.csv', 'df_noBase_noBase.csv']


In [3]:
tie_dic1 = {
    "TOOL_GROUP": ["Tool", "ChuckID"],
    "PR1_GROUP": ["Prev1Tool", "ChuckID"],
    "PR2_GROUP": ["Prev2Tool", "ChuckID"]
    }
tie_dic2 = {
    "TOOL_GROUP": ["Tool", "ChuckID"],
    "PR_GROUP": ["Prev1Tool", "Prev2Tool", "ChuckID"]
    }
tie_dic3 = {
    "ALL_GROUP": ["Tool", "Prev1Tool", "Prev2Tool", "ChuckID"]
    }

tie_dic_lst = [tie_dic1, tie_dic2, tie_dic3]
tie_dic_type_lst = ["Tool|Chuck, Pre1|Chuck, Pre2|Chuck","Tool|Chuck, Pre1|Pre2|Chuck", "Tool|Pre1|Pre2|Chuck"]

def _check_BASE(df_data, new_feat_lst):
    if "Prev2Tool|ChuckID" not in new_feat_lst:
        return
    else:
        cond_base = df_data["Prev2Tool|ChuckID"].str.contains("BASE")
        cond_c1 = df_data["Prev2Tool|ChuckID"].str.contains("#C1")
        cond_c2 = df_data["Prev2Tool|ChuckID"].str.contains("#C2")

        df_data["Prev2Tool|ChuckID"][cond_base & cond_c1] = "BASE"
        df_data["Prev2Tool|ChuckID"][cond_base & cond_c2] = "BASE"

_concat = lambda x: "|".join(x)
def combine_feat(df_data, tie_dic):
    ori_feat_lst = list(df_data.columns)
    new_feat_lst = list(df_data.columns)
    for feat_k, feat_v in tie_dic.items():
        _col_name = _concat(feat_v)
        new_feat_lst.append(_col_name)
        new_feat_lst = [i for i in new_feat_lst if i not in feat_v]
        df_data[_col_name] = df_data[feat_v].apply(_concat, axis=1)

    _check_BASE(df_data, new_feat_lst)

    return df_data, ori_feat_lst, new_feat_lst

def cols_split(df_raw):
    df_res = pd.DataFrame()
    cols_name = df_raw.columns
    df_data = df_raw.copy()
    
    for _col in cols_name:
        new_col = _col.split("|")
        # print(f'Column : {_col} => New Column : {new_col}')
        
        if _col != "Prev2Tool|ChuckID":
            _split_df = df_data[_col].str.split("|", expand=True)
            df_cur = pd.DataFrame(_split_df, copy=True)
            df_cur.columns = new_col
            df_res[df_cur.columns] = df_cur
        else:
            df_data_pre2_no_base = df_data[df_data[_col] != "BASE"]
            df_data_pre2_base = df_data[df_data[_col] == "BASE"]
            concat_lst = []
            
            _split_df_no_base = df_data_pre2_no_base[_col].str.split("|", expand=True)
            if _split_df_no_base.shape[0] > 0:
                cur_df_no_base = pd.DataFrame(_split_df_no_base, copy=True)
                cur_df_no_base.columns = new_col
                cur_df_no_base = cur_df_no_base.drop(columns=["ChuckID"])
                concat_lst.append(cur_df_no_base)
            
            _split_df_base = df_data_pre2_base[_col].str.split("|", expand=True)
            cur_df_base = pd.DataFrame(_split_df_base, copy=True)
            cur_df_base.columns = new_col[:_split_df_base.shape[1]]
            concat_lst.append(cur_df_base)
            
            cur_df = pd.concat(concat_lst)
            df_res[cur_df.columns] = cur_df

    return df_res

In [4]:
def cols_split_fail(df_raw):
    df_res = pd.DataFrame()
    cols_name = df_raw.columns
    df_data = df_raw.copy()

    for _col in cols_name:
        new_col = _col.split("|")
        print(f'Column : {_col} => New Column : {new_col}')

        _split_df = df_data[_col].str.split("|", expand=True)
        df_cur = pd.DataFrame(_split_df, copy=True)
        df_cur.columns = new_col
        
        if "Prev2Tool" in new_col and "ChuckID" in df_res.columns:
            df_cur = df_cur.drop(columns=["ChuckID"])
            
        df_res[df_cur.columns] = df_cur
    
    return df_res    

# Test

In [5]:
for test_data, test_data_name in zip(test_data_lst, test_data_name_lst):
    print(f'Testing {test_data_name}, Shape: {test_data.shape}')
    print(test_data.iloc[:20,:], end="\n\n")
    for tie_dic, tie_dic_type in zip(tie_dic_lst,tie_dic_type_lst):
        print(f'Testing Data: {test_data_name} | tie_dic: {tie_dic_type} ...', end="")
        df_data = test_data.copy()
        df_data, ori_feat_lst, new_feat_lst = combine_feat(df_data, tie_dic)
        df_data = df_data[new_feat_lst]

        df_res = cols_split(df_data)
        # df_res = cols_split_fail(df_data)
        assert test_data.equals(df_res[test_data.columns])
        print(f'\rTesting Data: {test_data_name} | tie_dic: {tie_dic_type} ... OK!')
    print("-" * 50, end="\n\n")

Testing df_mixed_mixed.csv, Shape: (72, 5)
   Part    Tool Prev1Tool Prev2Tool ChuckID
0    P1  TOOL_A     PR1_A      BASE     #C1
1    P1  TOOL_A     PR1_A      BASE     #C2
2    P1  TOOL_A     PR1_A     PR2_A     #C1
3    P1  TOOL_A     PR1_A     PR2_A     #C2
4    P1  TOOL_A     PR1_A     PR2_B     #C1
5    P1  TOOL_A     PR1_A     PR2_B     #C2
6    P1  TOOL_A     PR1_B      BASE     #C1
7    P1  TOOL_A     PR1_B      BASE     #C2
8    P1  TOOL_A     PR1_B     PR2_A     #C1
9    P1  TOOL_A     PR1_B     PR2_A     #C2
10   P1  TOOL_A     PR1_B     PR2_B     #C1
11   P1  TOOL_A     PR1_B     PR2_B     #C2
12   P1  TOOL_B     PR1_A      BASE     #C1
13   P1  TOOL_B     PR1_A      BASE     #C2
14   P1  TOOL_B     PR1_A     PR2_A     #C1
15   P1  TOOL_B     PR1_A     PR2_A     #C2
16   P1  TOOL_B     PR1_A     PR2_B     #C1
17   P1  TOOL_B     PR1_A     PR2_B     #C2
18   P1  TOOL_B     PR1_B      BASE     #C1
19   P1  TOOL_B     PR1_B      BASE     #C2

Testing Data: df_mixed_mixed.csv

# Test Fail

In [6]:
for test_data, test_data_name in zip(test_data_lst, test_data_name_lst):
    print(f'Testing {test_data_name}, Shape: {test_data.shape}')
    print(test_data.iloc[:20,:], end="\n\n")
    for tie_dic, tie_dic_type in zip(tie_dic_lst,tie_dic_type_lst):
        print(f'Testing Data: {test_data_name} | tie_dic: {tie_dic_type} ...', end="")
        df_data = test_data.copy()
        df_data, ori_feat_lst, new_feat_lst = combine_feat(df_data, tie_dic)
        df_data = df_data[new_feat_lst]

        # df_res = cols_split(df_data)
        df_res = cols_split_fail(df_data)
        assert test_data.equals(df_res[test_data.columns])
        print(f'\rTesting Data: {test_data_name} | tie_dic: {tie_dic_type} ... OK!')
    print("-" * 50, end="\n\n")

Testing df_mixed_mixed.csv, Shape: (72, 5)
   Part    Tool Prev1Tool Prev2Tool ChuckID
0    P1  TOOL_A     PR1_A      BASE     #C1
1    P1  TOOL_A     PR1_A      BASE     #C2
2    P1  TOOL_A     PR1_A     PR2_A     #C1
3    P1  TOOL_A     PR1_A     PR2_A     #C2
4    P1  TOOL_A     PR1_A     PR2_B     #C1
5    P1  TOOL_A     PR1_A     PR2_B     #C2
6    P1  TOOL_A     PR1_B      BASE     #C1
7    P1  TOOL_A     PR1_B      BASE     #C2
8    P1  TOOL_A     PR1_B     PR2_A     #C1
9    P1  TOOL_A     PR1_B     PR2_A     #C2
10   P1  TOOL_A     PR1_B     PR2_B     #C1
11   P1  TOOL_A     PR1_B     PR2_B     #C2
12   P1  TOOL_B     PR1_A      BASE     #C1
13   P1  TOOL_B     PR1_A      BASE     #C2
14   P1  TOOL_B     PR1_A     PR2_A     #C1
15   P1  TOOL_B     PR1_A     PR2_A     #C2
16   P1  TOOL_B     PR1_A     PR2_B     #C1
17   P1  TOOL_B     PR1_A     PR2_B     #C2
18   P1  TOOL_B     PR1_B      BASE     #C1
19   P1  TOOL_B     PR1_B      BASE     #C2

Testing Data: df_mixed_mixed.csv

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements